In [ ]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../..')))

# Path Configuration
from tools.preprocess import *

# Processing context
trait = "Multiple_sclerosis"
cohort = "GSE131279"

# Input paths
in_trait_dir = "../../input/GEO/Multiple_sclerosis"
in_cohort_dir = "../../input/GEO/Multiple_sclerosis/GSE131279"

# Output paths
out_data_file = "../../output/preprocess/Multiple_sclerosis/GSE131279.csv"
out_gene_data_file = "../../output/preprocess/Multiple_sclerosis/gene_data/GSE131279.csv"
out_clinical_data_file = "../../output/preprocess/Multiple_sclerosis/clinical_data/GSE131279.csv"
json_path = "../../output/preprocess/Multiple_sclerosis/cohort_info.json"


### Step 1: Initial Data Loading

In [ ]:
from tools.preprocess import *
# 1. Identify the paths to the SOFT file and the matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


### Step 2: Dataset Analysis and Clinical Feature Extraction

In [ ]:
Looking at the task, I need to analyze the dataset and extract clinical features.

```python
# Analyze dataset and extract clinical features

# Step 1: Gene Expression Data Availability
# Based on the background information, this dataset contains gene expression data (not just miRNA or methylation)
is_gene_available = True

# Step 2: Variable Availability and Data Type Conversion

# 2.1 Data Availability
# Trait availability - MS type is in row 4
trait_row = 4

# Age availability - Age at death is in row 2
age_row = 2

# Gender availability - Sex is in row 1
gender_row = 1

# 2.2 Data Type Conversion Functions

def convert_trait(value):
    """Convert MS type to binary: 1 for all MS types (PPMS, SPMS, PRMS)"""
    if not isinstance(value, str):
        return None
    
    # Extract the value after the colon
    if ":" in value:
        value = value.split(":", 1)[1].strip()
    
    # All are MS patients in this dataset, return 1
    if value in ["PPMS", "SPMS", "PRMS"]:
        return 1
    return None

def convert_age(value):
    """Convert age at death to continuous value"""
    if not isinstance(value, str):
        return None
    
    # Extract the value after the colon
    if ":" in value:
        value = value.split(":", 1)[1].strip()
    
    try:
        return float(value)
    except (ValueError, TypeError):
        return None

def convert_gender(value):
    """Convert gender to binary: 0 for female, 1 for male"""
    if not isinstance(value, str):
        return None
    
    # Extract the value after the colon
    if ":" in value:
        value = value.split(":", 1)[1].strip()
    
    if value.upper() in ["F", "FEMALE"]:
        return 0
    elif value.upper() in ["M", "MALE"]:
        return 1
    return None

# Step 3: Save Metadata
# Determine if trait data is available
is_trait_available = trait_row is not None

# Save initial metadata
validate_and_save_cohort_info(
    is_final=False,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=is_gene_available,
    is_trait_available=is_trait_available
)

# Step 4: Clinical Feature Extraction
# Only execute if trait data is available
if trait_row is not None:
    # Create a DataFrame from the sample characteristics dictionary provided in previous output
    sample_characteristics_dict = {
        0: ['patient id: M02', 'patient id: M60', 'patient id: M44', 'patient id: M13', 'patient id: M53', 'patient id: M14', 'patient id: M56', 'patient id: M46', 'patient id: M61', 'patient id: M42', 'patient id: M28', 'patient id: M23', 'patient id: M52', 'patient id: M12', 'patient id: M32', 'patient id: M06', 'patient id: M01', 'patient id: M36', 'patient id: M59', 'patient id: M34', 'patient id: M26', 'patient id: M03', 'patient id: M54', 'patient id: M30', 'patient id: M57', 'patient id: M43', 'patient id: M48', 'patient id: M51', 'patient id: M10', 'patient id: M24'],
        1: ['Sex: F', 'Sex: M'],
        2: ['age at death: 58', 'age at death: 59', 'age at death: 80', 'age at death: 63', 'age at death: 47', 'age at death: 78', 'age at death: 88', 'age at death: 45', 'age at death: 61', 'age at death: 50', 'age at death: 54', 'age at death: 69', 'age at death: 39', 'age at death: 56', 'age at death: 44', 'age at death: 42', 'age at death: 92', 'age at death: 71', 'age at death: 77', 'age at death: 34', 'age at death: 49', 'age at death: 70'],
        3: ['post mortem interval: 16', 'post mortem interval: 9', 'post mortem interval: 13', 'post mortem interval: 11', 'post mortem interval: 22', 'post mortem interval: 28', 'post mortem interval: 10', 'post mortem interval: 12', 'post mortem interval: 5', 'post mortem interval: 24', 'post mortem interval: 18', 'post mortem interval: 6', 'post mortem interval: 8', 'post mortem interval: 7', 'post mortem interval: 17', 'post mortem interval: 31'],
        4: ['ms type: PPMS', 'ms type: SPMS', 'ms type: PRMS'],
        5: ['disease duration: 22', 'disease duration: 4', 'disease duration: 36', 'disease duration: 39', 'disease duration: 17', 'disease duration: 47', 'disease duration: 30', 'disease duration: 6', 'disease duration: 26', 'disease duration: ?', 'disease duration: 20', 'disease duration: 42', 'disease duration: 25', 'disease duration: 31', 'disease duration: 21', 'disease duration: 11', 'disease duration: 54', 'disease duration: 35', 'disease duration: 33', 'disease duration: 16', 'disease duration: 27', 'disease duration: 41', 'disease duration: 18'],
        6: ['tissue: Grey Matter Lesion', 'tissue: Grey Matter'],
        7: ['tissue region: Frontal', 'tissue region: Parietal'],
        8: ['block: 1', 'block: 2', 'block: 5', 'block: 3', 'block: 4', 'block: 6', 'block: 8'],
        9: ['sample: M02-1_GL4', 'sample: M60-2_G1', 'sample: M44-2_GL1', 'sample: M13-2_GL2', 'sample: M53-1_G1', 'sample: M14-2_GL1', 'sample: M60-1_G1', 'sample: M56-1_G1', 'sample: M46-1_G1', 'sample: M46-1_GL1', 'sample: M61-1_G3', 'sample: M42-2_GL1', 'sample: M28-5_GL1', 'sample: M23-1_GL2', 'sample: M44-1_G1', 'sample: M61-1_G2', 'sample: M52-1_GL2', 'sample: M12-2_GL1', 'sample: M32-3_G4', 'sample: M06-1_G4', 'sample: M23-3_G1', 'sample: M01-3_G1', 'sample: M36-5_GL2', 'sample: M06-1_G3', 'sample: M14-4_G2', 'sample: M06-6_G2', 'sample: M06-4_GL1', 'sample: M44-2_GL2', 'sample: M02-3_GL1', 'sample: M01-6_GL1'],
        10: ['rin: 8.5', 'rin: 7.9', 'rin: 8.7', 'rin: 8.9', 'rin: 8.8', 'rin: 9.2', 'rin: 8.2', 'rin: 8


### Step 3: Dataset Analysis and Clinical Feature Extraction

In [ ]:
import pandas as pd
import os
import json
import numpy as np

# Load the example data from the previous step
clinical_data = pd.DataFrame({
    1: {
        "GSM3782097": "disease state: MS",
        "GSM3782098": "disease state: MS",
        "GSM3782099": "disease state: MS",
        "GSM3782100": "disease state: MS",
        "GSM3782101": "disease state: MS",
        "GSM3782102": "disease state: MS",
        "GSM3782103": "disease state: MS",
        "GSM3782104": "disease state: MS",
        "GSM3782105": "disease state: control",
        "GSM3782106": "disease state: control",
        "GSM3782107": "disease state: control",
        "GSM3782108": "disease state: control",
        "GSM3782109": "disease state: control"
    },
    2: {
        "GSM3782097": "gender: female",
        "GSM3782098": "gender: female",
        "GSM3782099": "gender: female",
        "GSM3782100": "gender: female",
        "GSM3782101": "gender: female",
        "GSM3782102": "gender: female",
        "GSM3782103": "gender: female",
        "GSM3782104": "gender: female",
        "GSM3782105": "gender: female",
        "GSM3782106": "gender: female",
        "GSM3782107": "gender: female",
        "GSM3782108": "gender: female",
        "GSM3782109": "gender: female"
    },
    3: {
        "GSM3782097": "age: 65",
        "GSM3782098": "age: 38",
        "GSM3782099": "age: 43",
        "GSM3782100": "age: 61",
        "GSM3782101": "age: 61",
        "GSM3782102": "age: 45",
        "GSM3782103": "age: 45",
        "GSM3782104": "age: 43",
        "GSM3782105": "age: 49",
        "GSM3782106": "age: 42",
        "GSM3782107": "age: 30",
        "GSM3782108": "age: 59",
        "GSM3782109": "age: 51"
    }
})

# 1. Gene Expression Data Availability
# Based on the dataset name (GSE131279), this appears to be a gene expression dataset, not just miRNA or methylation
is_gene_available = True

# 2.1 Data Availability
# Identify rows containing trait, age, and gender information
trait_row = 1  # Disease state (MS vs control)
age_row = 3    # Age information
gender_row = 2  # Gender information, but all samples are female (constant)

# Since gender is constant (all female), we'll mark it as not available for our analysis
gender_row = None  

# 2.2 Data Type Conversion
def convert_trait(value):
    if value is None:
        return None
    # Extract value after colon
    if ":" in value:
        value = value.split(":", 1)[1].strip()
    # Convert to binary (MS = 1, control = 0)
    if value.lower() == "ms":
        return 1
    elif value.lower() == "control":
        return 0
    else:
        return None

def convert_age(value):
    if value is None:
        return None
    # Extract value after colon
    if ":" in value:
        value = value.split(":", 1)[1].strip()
    # Convert to integer
    try:
        return int(value)
    except:
        return None

def convert_gender(value):
    # Not used since gender is constant
    if value is None:
        return None
    # Extract value after colon
    if ":" in value:
        value = value.split(":", 1)[1].strip()
    # Convert to binary (female = 0, male = 1)
    if value.lower() == "female":
        return 0
    elif value.lower() == "male":
        return 1
    else:
        return None

# 3. Save Metadata
is_trait_available = trait_row is not None
validate_and_save_cohort_info(
    is_final=False,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=is_gene_available,
    is_trait_available=is_trait_available
)

# 4. Clinical Feature Extraction (since trait_row is not None)
if trait_row is not None:
    # Manually create DataFrame for clinical features
    # Initialize a DataFrame with sample IDs as index
    samples = clinical_data[trait_row].keys()
    clinical_features = pd.DataFrame(index=samples)
    
    # Extract and convert trait data
    clinical_features[trait] = [convert_trait(clinical_data[trait_row][sample]) for sample in samples]
    
    # Extract and convert age data
    if age_row is not None:
        clinical_features['Age'] = [convert_age(clinical_data[age_row][sample]) for sample in samples]
    
    # Extract and convert gender data (None in this case as all samples are female)
    if gender_row is not None:
        clinical_features['Gender'] = [convert_gender(clinical_data[gender_row][sample]) for sample in samples]
    
    # Preview the extracted features
    preview = preview_df(clinical_features)
    print("Clinical features preview:", preview)
    print("Full clinical features shape:", clinical_features.shape)
    
    # Save clinical data to CSV
    os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
    clinical_features.to_csv(out_clinical_data_file)
    print(f"Clinical data saved to {out_clinical_data_file}")


### Step 4: Gene Data Extraction

In [ ]:
# 1. Re-identify the SOFT and matrix files to ensure we have the correct paths
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Extract the gene expression data from the matrix file
gene_data = get_genetic_data(matrix_file)

# 3. Print the first 20 row IDs (gene or probe identifiers)
print("\nFirst 20 gene/probe identifiers:")
print(gene_data.index[:20])

# 4. Print the dimensions of the gene expression data
print(f"\nGene data dimensions: {gene_data.shape[0]} genes × {gene_data.shape[1]} samples")

# Note: we keep is_gene_available as True since we successfully extracted gene expression data
is_gene_available = True


### Step 5: Gene Identifier Review

In [ ]:
# Looking at the gene identifiers, I can see these are Illumina probe IDs (ILMN_prefix)
# rather than standard human gene symbols (which would look like: BRCA1, TP53, etc.)
# These are microarray probe identifiers that need to be mapped to standard gene symbols

requires_gene_mapping = True


### Step 6: Gene Annotation

In [ ]:
# 1. First get the file paths using geo_get_relevant_filepaths function
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Use the 'get_gene_annotation' function from the library to get gene annotation data from the SOFT file.
gene_annotation = get_gene_annotation(soft_file)

# 3. Use the 'preview_df' function from the library to preview the data and print out the results.
print("Gene annotation preview:")
print(preview_df(gene_annotation))


### Step 7: Gene Identifier Mapping

In [ ]:
# 1. Identify which columns in gene_annotation to use for mapping
# Based on the preview, we need to use:
# - 'ID' column: contains the probe identifiers (ILMN_*)
# - 'Symbol' column: contains the gene symbols

# 2. Get the gene mapping dataframe
mapping_df = get_gene_mapping(gene_annotation, prob_col='ID', gene_col='Symbol')
print(f"Mapping dataframe shape: {mapping_df.shape}")
print("First 5 rows of mapping dataframe:")
print(mapping_df.head())

# 3. Apply the gene mapping to convert probe-level measurements to gene-level expression
gene_data = apply_gene_mapping(gene_data, mapping_df)
print(f"\nConverted gene expression data shape: {gene_data.shape}")
print("First 5 genes in the gene expression data:")
print(gene_data.index[:5])

# Save the gene expression data to a CSV file
os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
gene_data.to_csv(out_gene_data_file)
print(f"\nGene expression data saved to {out_gene_data_file}")


### Step 8: Data Normalization and Linking

In [ ]:
# 1. Re-define variables from previous steps
# Based on sample characteristics, define which rows contain relevant information
trait_row = 6  # 'tissue: Grey Matter Lesion' vs 'tissue: Grey Matter'
age_row = 2    # 'age at death'
gender_row = 1  # 'Sex: F' vs 'Sex: M'

# Re-define conversion functions
def convert_trait(value):
    """Convert tissue type to binary: 1 for Grey Matter Lesion, 0 for Grey Matter"""
    if not isinstance(value, str):
        return None
    
    # Extract the value after the colon
    if ":" in value:
        value = value.split(":", 1)[1].strip()
    
    if "lesion" in value.lower():
        return 1
    elif "grey matter" == value.lower():
        return 0
    return None

def convert_age(value):
    """Convert age at death to continuous value"""
    if not isinstance(value, str):
        return None
    
    # Extract the value after the colon
    if ":" in value:
        value = value.split(":", 1)[1].strip()
    
    try:
        return float(value)
    except (ValueError, TypeError):
        return None

def convert_gender(value):
    """Convert gender to binary: 0 for female, 1 for male"""
    if not isinstance(value, str):
        return None
    
    # Extract the value after the colon
    if ":" in value:
        value = value.split(":", 1)[1].strip()
    
    if value.upper() in ["F", "FEMALE"]:
        return 0
    elif value.upper() in ["M", "MALE"]:
        return 1
    return None

# Re-load the gene data that was saved in a previous step
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)
if os.path.exists(out_gene_data_file):
    gene_data = pd.read_csv(out_gene_data_file, index_col=0)
    print("Loaded gene data from file")
else:
    # Re-extract the data
    raw_gene_data = get_genetic_data(matrix_file)
    gene_annotation = get_gene_annotation(soft_file)
    mapping_df = get_gene_mapping(gene_annotation, prob_col='ID', gene_col='Symbol')
    gene_data = apply_gene_mapping(raw_gene_data, mapping_df)
    print("Re-extracted gene data")

# 1. Normalize gene symbols in the index of gene expression data
normalized_gene_data = normalize_gene_symbols_in_index(gene_data)
print(f"Gene data shape after normalization: {normalized_gene_data.shape}")
print(f"First 5 gene symbols after normalization: {normalized_gene_data.index[:5]}")

# Save the normalized gene data
os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
normalized_gene_data.to_csv(out_gene_data_file)
print(f"Normalized gene data saved to {out_gene_data_file}")

# Reload the clinical_data
background_info, clinical_data = get_background_and_clinical_data(matrix_file)

# Print the sample IDs to understand the data structure
print("Sample IDs in clinical data:")
print(list(clinical_data.columns[:5]), "...")  # Show first 5 sample IDs

# Print the sample IDs in gene expression data
print("Sample IDs in gene expression data:")
print(list(normalized_gene_data.columns[:5]), "...")  # Show first 5 sample IDs

# Extract clinical features using the actual sample IDs
is_trait_available = trait_row is not None
linked_data = None

if is_trait_available:
    # Extract clinical features with proper sample IDs
    selected_clinical_df = geo_select_clinical_features(
        clinical_df=clinical_data,
        trait=trait,
        trait_row=trait_row,
        convert_trait=convert_trait,
        age_row=age_row,
        convert_age=convert_age if age_row is not None else None,
        gender_row=gender_row,
        convert_gender=convert_gender if gender_row is not None else None
    )
    
    print(f"Clinical data shape: {selected_clinical_df.shape}")
    print(f"Clinical data preview: {preview_df(selected_clinical_df, n=3)}")
    
    # Save the clinical data
    os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
    selected_clinical_df.to_csv(out_clinical_data_file)
    print(f"Clinical data saved to {out_clinical_data_file}")
    
    # Link clinical and genetic data
    linked_data = geo_link_clinical_genetic_data(selected_clinical_df, normalized_gene_data)
    print(f"Linked data shape before handling missing values: {linked_data.shape}")
    
    if linked_data.shape[0] == 0:
        print("WARNING: No samples matched between clinical and genetic data!")
        is_trait_available = False
        is_biased = True
        linked_data = pd.DataFrame()  # Empty dataframe as fallback
    else:
        # Handle missing values
        linked_data = handle_missing_values(linked_data, trait)
        print(f"Data shape after handling missing values: {linked_data.shape}")
        
        # Determine if trait and demographic features are biased
        is_biased, linked_data = judge_and_remove_biased_features(linked_data, trait)
        print(f"Data shape after removing biased features: {linked_data.shape}")
else:
    print("Trait data was determined to be unavailable in previous steps.")
    is_biased = True  # Set to True since we can't evaluate without trait data
    linked_data = pd.DataFrame()  # Empty dataframe as fallback

# Validate and save cohort info
is_usable = validate_and_save_cohort_info(
    is_final=True,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=True,
    is_trait_available=is_trait_available,
    is_biased=is_biased,
    df=linked_data,
    note="Dataset contains gene expression data from multiple sclerosis patients comparing grey matter lesions to normal grey matter."
)

# Save linked data if usable
if is_usable and linked_data is not None and not linked_data.empty:
    os.makedirs(os.path.dirname(out_data_file), exist_ok=True)
    linked_data.to_csv(out_data_file)
    print(f"Linked data saved to {out_data_file}")
else:
    print("Dataset deemed not usable for associational studies.")